In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gpxpy
import ggps
import haversine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from gps_analysis import geodesy, files, splits, garmin

In [ ]:
positions = files.load_gpx("activity_768041870.gpx")
locations = pd.read_json('cam_locations.json', orient='index')
positions

In [ ]:
splits.find_all_best_times(positions).applymap(splits.strfmtsplit)

In [ ]:
splits.find_best_times(positions, 0.5).applymap(splits.strfmtsplit)

In [ ]:
loc_timings = splits.get_location_timings(positions, locations)
loc_timings.loc[('times', 3), ('splits', 3)].applymap(splits.strfsplit)

In [ ]:
i, j = np.triu_indices(len(positions), 1)
i, j

time_diffs = (
    positions.timeElapsed[j].dt.total_seconds().values - 
    positions.timeElapsed[i].dt.total_seconds().values)
time_diffs
dist_diffs =(
    positions.distance[j].values - 
    positions.distance[i].values)
time_diffs
speeds = 1000 * np.nan_to_num(dist_diffs / time_diffs)

In [ ]:
best_times.groupby('length').split.min()

In [ ]:
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

In [ ]:
mask = is_pareto_efficient(np.c_[-dist_diffs, -speeds])

plt.scatter(
    dist_diffs[mask],
    speeds[mask]
)

In [ ]:
plt.plot(positions.longitude, positions.latitude)

In [ ]:
api = garmin.login("imogen@gtsoftware.co.uk")

In [ ]:
activities = garmin.get_activities(0, 1)

In [ ]:
positions = garmin.load_fit_activity(activities.activityId[0])
positions

In [ ]:
locations = splits.load_locations()
locations

In [ ]:
pd.concat?

In [ ]:
pd.read_csv("data/ely_locations.csv").to_csv(
    "data/ely_locations.tsv", sep='\t', index=False
)

In [ ]:
pd.concat([
    pd.read_csv("data/ely_locations.csv", index_col=0),
    pd.read_csv("data/cam_locations.csv", index_col=0),
    locations
])

In [ ]:
cav_locations = {
    'start': [51.461398172524824, -0.9455592979289794],
    'finish': [51.47420259934778, -0.9253479107461384],
}

In [ ]:
locations = {
    'start': [51.461398172524824, -0.9455592979289794],
    '500m': [51.46460471321316, -0.9404994529495201],
    '1km': [51.467811036115336, -0.9354388968526993],
    '1.5km': [51.47101714117572, -0.930377629505215], 
    'finish': [51.47420259934778, -0.9253479107461384],
}
locations = pd.DataFrame.from_dict({
    k: {
        'latitude': v[0], 
        'longitude': v[1],
        'bearing': geodesy.estimate_bearing(
            positions, geodesy.LatLon(*v), 0.05)
    } 
    for k, v in locations.items()
},
    orient='index')
locations.index.name='location'
locations.bearing = geodesy.bearing(
    locations.loc['start'],
    locations.loc['finish']
)

In [ ]:
locations.to_csv("data/cav_locations.tsv", sep='\t')

In [ ]:
timings = splits.get_location_timings(positions)
timings.applymap(splits.strfsplit).to_excel("timings.xlsx")

In [ ]:
splits.find_all_crossing_times(positions)

In [ ]:
locations.bearing = geodesy.bearing(
    locations.loc['start'],
    locations.loc['finish']
)

In [ ]:
geodesy.haversine_km(locations, locations.shift())

In [ ]:
geodesy.haversine_km(
    locations,
    start 
)

In [ ]:
start = geodesy.follow_bearing(
    locations.loc['finish'],
    -2.
).to_latlon()
tuple(start)

In [ ]:
pos = geodesy.follow_bearing(
    locations.loc['start'],
    1.5
).to_latlon()
tuple(pos)

In [ ]:
geodesy.bearing(
    locations.loc['start'],
    locations.loc['finish'])

In [ ]:
cav_locations = {
    'start': [51.46108381769548, -0.9450516671209354],
    'finish':
}

In [ ]:
plt.figure()
plt.plot(positions.longitude, positions.latitude)

plt.show()

In [ ]:
actid = 8026393665
actid = 2366290503

positions = garmin.load_activity(actid, api=api)

In [ ]:
garmin.login('imogen@gtsoftware.co.uk')
actid = 2366290503
data = garmin.load_fit_activity(actid)

In [ ]:
data.time.diff(-1)

In [ ]:
best_splits = splits.find_all_best_times(data)

In [ ]:
best_splits.loc["2km"]

In [ ]:
def interval_average_value(locations, label, start_dist, dist):
    interval = [start_dist, start_dist + dist]
    i0, i1 = locations.distance.searchsorted(interval)
    val = locations.loc[i0:i1, label]
    times = locations.timeElapsed.dt.total_seconds().copy()
    times.loc[[i0, i1 + 1]] = np.interp(interval, locations.distance, times)
    weights = -times.loc[i0:i1+1].diff(-1)[:-1]
    return np.average(val, weights=weights)

interval_average_value(data, 'cadence', 12.041, 2)

In [ ]:
interval_average_value(data, 'cadence', 3.4, 2)

In [ ]:
data.distance.diff(-1)

In [ ]:
plt.figure()
plt.plot(data.distance, data.cadence)

In [ ]:
distance = 2
dist_splits = best_splits.loc['2km']

plt.figure()
plt.plot(data.distance, 500/data.speed)
plt.plot(data.distance, data.heart_rate)
plt.plot(data.distance, data.cadence)

for d, s in dist_splits.split.iteritems():
    ts = s.total_seconds()
    plt.plot([d, d+distance], [ts, ts])
    
plt.ylim(0, 250)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)
garmin.activity_data_to_excel(activities, activity_data, api=api)

In [ ]:
activities = garmin.get_activities(activityType='rowing', api=api)
activity_data, errors = garmin.load_activities(activities.activityId, api=api)